In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [ ]:
root_folder = '../build/out/04_flips'
meshestxt = root_folder + '/plots/meshes.txt'

In [ ]:
run_names = [l[:-1] for l in open(meshestxt, "r").readlines()]

# Load Metrics

In [ ]:
# CELL-BASED TRIANGULATION METRICS
run_metrics = {}
run_orbitpoints = {}
for run_name in run_names:
    metrics_file     = os.path.join(root_folder, '{}metrics.csv'.format(run_name))
    orbitpoints_file = os.path.join(root_folder, '{}orbitpoints.csv'.format(run_name))
    
    # print(os.path.exists(metrics_file), os.path.exists(orbitpoints_file))
    run_metrics[run_name] = pd.read_csv(metrics_file)
    run_orbitpoints[run_name] = pd.read_csv(orbitpoints_file)

In [ ]:
# HEAT VALUES
run_data = {}
for r_idx, run_name in enumerate(run_names):
    # print('{}/{}'.format(r_idx+1, len(run_names)))
    _, cellSize, rerat, facetSize, lloyd, perturb, exude, minorbitpoints, nflips, _ = run_name.split('_')
    if (minorbitpoints == 0):
        print('No orbitpoints, no heatvals, skip...')
        continue
    try:
        #print('-----------')
        csv_path = os.path.join(root_folder, 'plots/{}heatvals.csv'.format(run_name))
        # print(csv_path)
        
        origin_index  = pd.read_csv(csv_path, header=None, skiprows=1, nrows=1).values[0][0]
        orbit_indices = list(pd.read_csv(csv_path, header=None, skiprows=3, nrows=1).values[0])
        #print(run_orbitpoints[run_name]['orbit_idx'].sort_values().tolist()[1:])
        #print(orbit_indices)
        # print(list(pd.read_csv(csv_path, header=None, skiprows=3, nrows=1)))
        heat_values   = pd.read_csv(csv_path, skiprows=4)
        orbit_heat_vals = heat_values.iloc[run_orbitpoints[run_name]['orbit_idx'].sort_values().tolist()[1:]] # heat_values.iloc[orbit_indices]
        run_data[run_name] = orbit_heat_vals
    except:
        print("heat falue file for {} missing".format(run_name))

# CELL-BASED METRICS

In [ ]:
from utils import join_run_values
metric_df = join_run_values(run_metrics, run_orbitpoints, run_names)
display(metric_df.groupby('nflips').mean())

### Metrics BY

In [ ]:
b_var = 'nflips'
avg_by_size = metric_df.groupby(b_var).min().sort_index()

fig = plt.figure(figsize=(15, 4))
axes = fig.subplots(nrows=1, ncols=3)
avg_by_size.plot(y='minangle', ax=axes[0])
avg_by_size.plot(y='amips',    ax=axes[1])
avg_by_size.plot(y='volume',   ax=axes[2])
plt.show()

# HEAT VALUES

In [ ]:
b_var = 'nflips'
df = join_run_values(run_data, run_orbitpoints, run_names)

fig = plt.figure(figsize=(10, 4))
axes = fig.subplots(nrows=1, ncols=2)
df.groupby(b_var)[['h_fem', ' h_dec']].mean().plot(logy=True, ax=axes[0])
axes[0].set_title('Mean')
df.groupby(b_var)[['h_fem', ' h_dec']].var().plot(logy=True, ax=axes[1])
axes[1].set_title('Var')
plt.show()

In [ ]:
print()

In [ ]:
for nf in list(df['nflips'].unique()):
    df[df['nflips'] == nf][['h_fem', ' h_dec']].boxplot()
    plt.title('{} Flips'.format(nf))
    plt.show()